In [1]:
import numpy as np
import plotly.graph_objects as go

np.random.seed(7)


In [2]:
n_assets = 5
mean_returns = np.array([0.08, 0.10, 0.12, 0.07, 0.09])
A = np.random.randn(n_assets, n_assets)
cov = A @ A.T
cov = cov / np.max(cov) * 0.04

n_portfolios = 5000
weights = np.random.random((n_portfolios, n_assets))
weights = weights / weights.sum(axis=1, keepdims=True)

portfolio_returns = weights @ mean_returns
portfolio_vol = np.sqrt(np.einsum('ij,jk,ik->i', weights, cov, weights))
risk_free = 0.02
sharpe = (portfolio_returns - risk_free) / portfolio_vol

max_sharpe_idx = np.argmax(sharpe)
min_vol_idx = np.argmin(portfolio_vol)


In [3]:
bins = np.linspace(portfolio_returns.min(), portfolio_returns.max(), 25)
frontier_r = []
frontier_v = []
for i in range(len(bins) - 1):
    mask = (portfolio_returns >= bins[i]) & (portfolio_returns < bins[i + 1])
    if np.any(mask):
        idx = np.argmin(portfolio_vol[mask])
        sel_vol = portfolio_vol[mask][idx]
        sel_ret = portfolio_returns[mask][idx]
        frontier_v.append(sel_vol)
        frontier_r.append(sel_ret)
frontier_v = np.array(frontier_v)
frontier_r = np.array(frontier_r)


In [4]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=portfolio_vol,
    y=portfolio_returns,
    mode='markers',
    marker=dict(color=sharpe, colorscale='Viridis', showscale=True),
    name='Portfolios'
))
fig.add_trace(go.Scatter(
    x=frontier_v,
    y=frontier_r,
    mode='lines+markers',
    name='Approx Frontier'
))
fig.add_trace(go.Scatter(
    x=[portfolio_vol[max_sharpe_idx]],
    y=[portfolio_returns[max_sharpe_idx]],
    mode='markers',
    marker=dict(color='red', size=10),
    name='Max Sharpe'
))
fig.add_trace(go.Scatter(
    x=[portfolio_vol[min_vol_idx]],
    y=[portfolio_returns[min_vol_idx]],
    mode='markers',
    marker=dict(color='black', size=9),
    name='Min Vol'
))
fig.update_layout(
    title='Efficient Frontier (Sampled)',
    xaxis_title='Volatility',
    yaxis_title='Expected Return',
    template='plotly_white'
)
fig.show()
